In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

In [7]:
df.head()

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-04-30,61870.0,245000.0,40.723982,130.528256,22.0,29.0,56.0,69.0,89.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202421.064584,Atlanta-Sandy Springs-Alpharetta
2,2012-05-31,125500.0,217450.0,63.913043,119.919216,24.0,40.0,63.0,60.0,144.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202681.309539,Atlanta-Sandy Springs-Alpharetta
3,2012-06-30,153000.0,189900.0,81.598080,105.617353,34.0,46.0,50.0,57.0,126.0,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202998.603897,Atlanta-Sandy Springs-Alpharetta
4,2012-07-31,165500.0,154000.0,81.598080,83.921175,39.0,49.0,42.0,50.0,80.0,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,203781.903446,Atlanta-Sandy Springs-Alpharetta


In [12]:
df.describe()

,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,...,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price
count,8.840920e+05,8.840920e+05,884092.000000,884092.000000,884092.000000,884092.000000,884092.000000,884092.000000,884092.000000,884092.000000,...,884092.000000,884092.000000,884092.000000,8.840920e+05,884092.000000,884092.000000,884092.000000,884092.000000,884092.000000,8.840920e+05
mean,3.941026e+05,4.229849e+05,223.068328,231.169836,76.722786,81.471457,92.910352,77.300772,61.289875,0.978253,...,21457.525088,8714.480844,1146.686284,3.147922e+05,11455.779512,829.769133,20998.337745,20998.337745,9687.919963,3.913289e+05
std,3.815481e+05,1.899201e+06,696.724208,290.120210,76.698331,85.327617,92.695852,89.041772,82.220497,0.063550,...,19554.169658,7588.635086,547.236746,2.672194e+05,10429.516147,954.753745,19008.391216,19008.391216,8841.743137,3.445383e+05
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,1.046432e+04
25%,1.850000e+05,1.990000e+05,110.639667,116.818175,19.000000,17.000000,24.000000,20.000000,26.000000,0.965158,...,4404.000000,1930.000000,852.000000,1.436000e+05,2320.000000,127.000000,4369.000000,4369.000000,1945.000000,1.897063e+05
50%,3.025000e+05,3.200000e+05,166.093929,173.142587,55.000000,57.000000,67.000000,50.000000,45.000000,0.982062,...,17489.000000,7426.000000,1114.000000,2.501000e+05,9299.000000,538.000000,17247.000000,17247.000000,7830.500000,3.050190e+05
75%,4.800000e+05,4.999000e+05,260.626280,270.181473,111.000000,119.000000,133.000000,103.000000,74.000000,0.998206,...,32991.000000,13564.000000,1446.000000,4.125000e+05,17702.000000,1192.000000,32290.000000,32290.000000,15013.000000,4.797111e+05
max,2.050000e+07,1.000000e+09,366700.000000,143015.398551,955.000000,1374.000000,1112.000000,1941.000000,7777.000000,1.906250,...,130605.000000,48734.000000,3501.000000,2.000001e+06,68735.000000,9735.000000,126948.000000,126948.000000,60956.000000,8.463116e+06
